Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [81]:
from collections import namedtuple
from random import choice,seed
from tqdm.auto import tqdm
import numpy as np
from icecream import ic
import heapq
import math

#SEED = 1897  funziona

SEED = 43242# molto meglio
#SEED=4

seed(SEED)

In [82]:
PUZZLE_DIM = 4
action = namedtuple('Action', ['pos1', 'pos2'])

objective=[]
Y=PUZZLE_DIM
X=PUZZLE_DIM
for i in range(Y):
    locobjective=[]
    for k in range(X):
        locobjective.append(k+1+i*X)
    
    objective.append(locobjective)
objective[-1][-1]=0 
""" objective=[[ 1,  2,  3,  4],
           [ 12,  13,  14,  5],
           [ 11,  0,  15,  6],
           [ 10,  9,  8,  7],]
 """

objective=np.array(objective, dtype=np.int8)
objective

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15,  0]], dtype=int8)

In [83]:
def is_solvable(puzzle, goal):
    puzzle_flat = puzzle.flatten()
    goal_flat = goal.flatten()

    goal_positions = {value: idx for idx, value in enumerate(goal_flat)}
    puzzle_mapped = [goal_positions[val] for val in puzzle_flat if val != 0]

    inversions = 0
    for i in range(len(puzzle_mapped)):
        for j in range(i + 1, len(puzzle_mapped)):
            if puzzle_mapped[i] > puzzle_mapped[j]:
                inversions += 1
    
    height, width = puzzle.shape
    blank_row_from_bottom = height - np.where(puzzle == 0)[0][0]
    # ic(blank_row_from_bottom)

    if width % 2 == 1:  # Odd-sized puzzle
        return inversions % 2 == 0
    else:  # Even-sized puzzle
        return (blank_row_from_bottom % 2 == 0) == (inversions % 2 == 0)



def available_actions(state: np.ndarray,width,hight) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < hight - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < width - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [84]:
RANDOMIZE_STEPS =100003#100_000

state = np.empty_like(objective)
solfinal=np.empty_like(objective)
k=0
state=objective[:] #np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    if(k==5):
        ic("fuiewf")
    state = do_action(state, choice(available_actions(state,X,Y)))
    k+=1
state

Randomizing:   0%|          | 0/100003 [00:00<?, ?it/s]

ic| 'fuiewf'


array([[15, 11, 14, 12],
       [ 2,  7,  6,  5],
       [13,  0, 10,  9],
       [ 8,  4,  1,  3]], dtype=int8)

In [ ]:
#state2 = do_action(state, available_actions(state)[0])
ic(state)
#ic(objective)


def distanceEuclidean(state,objective):
    distance =0


    for value in range(PUZZLE_DIM*PUZZLE_DIM):
        # Find the position of the element in both arrays
        pos_state = np.argwhere(state == value)[0]
        pos_objective = np.argwhere(objective == value)[0]
        
        # Compute the Euclidean distance between the positions
        distance += np.sqrt(np.sum((pos_state - pos_objective) ** 2))
    return distance
    

def distanceManhattan(state, objective,width,hight):
    distance = 0
    
    # Iterate over each unique element in the arrays (assuming numbers from 1 to PUZZLE_DIM**2 - 1)
    for i in range(0, (width*hight)):
        value=pos_objective[i//hight][i%hight]
        # Find the position of the element in both arrays
        pos_state = np.argwhere(state == value)[0]
        pos_objective = np.argwhere(objective == value)[0]
        
        # Compute the Manhattan distance between the positions
        distance += np.sum(np.abs(pos_state - pos_objective))
    
    return distance


def distanceManhattan2(state, objective,y,x):
    distance = 0
    rightpos=0
    
    # Iterate over each unique element in the arrays (assuming numbers from 1 to PUZZLE_DIM**2 - 1)
    for value in range(0, PUZZLE_DIM**2):
        # Find the position of the element in both arrays
        pos_state = np.argwhere(state == value)[0]
        pos_objective = np.argwhere(objective == value)[0]
        
        # Compute the Manhattan distance between the positions
        dif=np.sum(np.abs(pos_state - pos_objective))
        
        if(dif==0):
            rightpos+=1
        else:
            distance += dif


    distance=distance+(PUZZLE_DIM**2-rightpos)*2
    return distance

def linear_conflict_distance(curr, goal,y,x,start=0):
    distance = 0
    linear_conflict = 0
    goodfirstline=[1,2,3,4,5,9,13]
  
    for i in range(y):
        for j in range(x):
            #if curr[i][j] != 0:
            if (start==1 and curr[i][j] in goodfirstline) or (curr[i][j] != 0 and start==0):
                # Compute Manhattan distance
                goal_i, goal_j = np.where(goal == curr[i][j])
                distance += abs(goal_i - i) + abs(goal_j - j)
                # Check for linear conflicts in the row
                if goal_i == i:
                    for k in range(j + 1, x):  # Compare with other tiles in the row
                        if curr[i][k] != 0: 
                            goal_k_i, goal_k_j = np.where(goal == curr[i][k])
                            if goal_k_i == i and goal_j > goal_k_j:
                                linear_conflict += 2
                # Check for linear conflicts in the column
                if goal_j == j:
                    for k in range(i + 1, y):  # Compare with other tiles in the column
                        if curr[k][j] != 0:
                            goal_k_i, goal_k_j = np.where(goal == curr[k][j])
                            if goal_k_j == j and goal_i > goal_k_i:
                                linear_conflict += 2 

    return (distance[0] + linear_conflict)


def discard_n_worst(open_list, n):
    if n <= 0:
        return open_list  # No need to discard anything
    if len(open_list) <= n:
        return []  # Discard all if `n` is greater than or equal to the queue size
    
    # Extract all elements and sort them by `f` value
    all_elements = sorted(open_list, key=lambda x: x[0])  # Sort by `f` value (first tuple element)
    
    # Keep only the best (smallest `f` values)
    remaining_elements = all_elements[:-n]  # Remove the last `n` elements
    
    # Rebuild the heap
    heapq.heapify(remaining_elements)
    return remaining_elements

def check_if_row_or_col_eq(state, objective):
    completed=(0,0)

    if np.array_equal(state[0], objective[0]):
        completed=(completed[0]+1,completed[1])
    if np.array_equal(state[:,0], objective[:,0]):
        completed=(completed[0],completed[1]+1)

    
    return completed


def astar(state,globobjective,distancefun):
    xsolfinal=0
    ysolfinal=0
    cost=0
    maxsize=1000000
    open_list = []
    globcost=0
    tuplestate=tuple(state)
    hashable_state = tuple(tuple(arr) for arr in state)
    heapq.heappush(open_list, (0, hashable_state))
 
    visited = set()
    target_size=math.factorial(PUZZLE_DIM*PUZZLE_DIM)

    heuristic_cache = {}
    objective=globobjective[:]
    hight=Y
    w=X
    if(X>=4 and Y>=4):
        firstcut=1
    else:
        firstcut=0
    
    heuristic_cache[hashable_state] = distancefun(state, objective,hight,w,firstcut)
    
    analized=0
    analized+=1





    open_list_dict = {} 
    #with tqdm(total=target_size, desc="Filling the set", unit="item") as pbar:
    if 1:
        lenopen=len(open_list)

        while  lenopen>0:   #
            current_cost, current_state = heapq.heappop(open_list)
            lenopen-=1
            if current_state in visited:
                
                continue
            
            listcurrentstate=np.array(current_state, dtype=np.int8)
            #
            if(lenopen%1000000==0):
               ic(listcurrentstate)
              
            if (hight>3 and w>3):
                to_del=check_if_row_or_col_eq(listcurrentstate,objective)
                if to_del!=(0,0):
                    #ic(listcurrentstate)
                    solvable=0
                    if(to_del[0]==1):
                        listcurrentstatetemp=listcurrentstate[1:,:]
                        objectivetemp=objective[1:,:]
                        if   1:#is_solvable(listcurrentstatetemp,objectivetemp):
                            solvable=1
                            objective=objective[1:,:]
                            solfinal[ysolfinal, xsolfinal:xsolfinal + listcurrentstate.shape[1]] = listcurrentstate[0, :]
                            ysolfinal += 1
                            #solfinal[ysolfinal,xsolfinal:]=listcurrentstate[1:,:]
                            listcurrentstate=listcurrentstate[1:,:]
                            hight=hight-1
                    
                    if(to_del[1]==1):
                        listcurrentstatetemp=listcurrentstate[:,1:]
                        objectivetemp=objective[:,1:]
                        if 1:#is_solvable(listcurrentstatetemp,objectivetemp):
                            solvable=1
                            objective=objective[:,1:]



                            solfinal[ysolfinal:ysolfinal + listcurrentstate.shape[0], xsolfinal] = listcurrentstate[:, 0]
                            xsolfinal += 1
                            #solfinal[ysolfinal:,xsolfinal]=listcurrentstate[1:,:]
                            listcurrentstate=listcurrentstate[:,1:]
                            w-=1
                    
                    if(solvable!=0):
                        ic("ho modificato la forma!!!")
                        open_list = []
                        hashable_state = tuple(tuple(arr) for arr in listcurrentstate)
                        #heapq.heappush(open_list, (0, hashable_state))
                        visited = set()
                        lenopen=0
                        firstcut=0
                        heuristic_cache = {}
                        heuristic_cache[hashable_state] = distancefun(listcurrentstate, objective,hight,w,firstcut)
                        analized+=1
                        current_cost=current_cost
                        current_state=hashable_state






                    ic(listcurrentstate) 
            
            if np.array_equal(listcurrentstate, objective) == True:
                ic(analized)
                """    if check_if_row_or_col_eq(listcurrentstate,objective)!=0:
                    ic(listcurrentstate) """
                solfinal[ysolfinal:ysolfinal + listcurrentstate.shape[0],xsolfinal: xsolfinal+ listcurrentstate.shape[1]] = listcurrentstate[:,:]

                return solfinal, current_cost,analized
            
            
            #dim=len(open_list)
            #ic(dim)
            
            """  if len(open_list) > maxsize*10:
                open_list = discard_n_worst(open_list, 1) """
            
            

            #flen=len(visited)
            visited.add(current_state)  ## controlla


            """ seclen=(len(visited))
            if( flen!=seclen):
                 pbar.update(1) """

            actions=available_actions(listcurrentstate,w,hight)
        
            #hcosts=np.zeros(len(actions))
            for ia in range(len(actions)):
                #hcosts[ia]=distancefun(do_action(state,actions[ia]),objective)
                newstate=do_action(listcurrentstate,actions[ia])
                #analized+=1
                hashable_new_state = tuple(tuple(arr) for arr in newstate)
                if hashable_new_state not in visited:
                    g = current_cost + 1  # Assume cost per step is 1; adjust if needed
                    #h = distancefun(newstate, objective)
                    #state_key = state_to_string(newstate)
                    if hashable_new_state not in heuristic_cache:
                        heuristic_cache[hashable_new_state] = distancefun(newstate, objective,hight,w,firstcut)
                        analized+=1
                    h = heuristic_cache[hashable_new_state]
                    f = float(g + h)

                    if hashable_new_state in open_list_dict and open_list_dict[hashable_new_state] <= f:
                       
                        continue  # Skip adding this state 
                    
                    # Update the best f value for this state
                    open_list_dict[hashable_new_state] = f
                    heapq.heappush(open_list, (f, hashable_new_state))
                    lenopen+=1
    
        #ic(listcurrentstate)


        


    




#ris=distanceEuclidean(state,objective)
#ris2=distanceManhattan(state,objective)

ic(is_solvable(state,objective))
state,cost,analized=astar(state,objective,linear_conflict_distance)  # xe uso euclidea non funziona perche non è ammissibile (penso)
#ic(ris2)
ic(cost)
ic(state)
ic(analized)
    

ic| state: array([[15, 11, 14, 12],
                  [ 2,  7,  6,  5],
                  [13,  0, 10,  9],
                  [ 8,  4,  1,  3]], dtype=int8)
ic| is_solvable(state,objective): np.False_
ic| listcurrentstate: array([[15, 11, 14, 12],
                             [ 2,  7,  6,  5],
                             [13,  0, 10,  9],
                             [ 8,  4,  1,  3]], dtype=int8)
ic| listcurrentstate: array([[15, 14,  5, 12],
                             [ 2, 11,  6,  0],
                             [13,  7,  3,  9],
                             [ 8, 10,  4,  1]], dtype=int8)
ic| listcurrentstate: array([[15, 11, 14, 12],
                             [ 2,  0,  7,  9],
                             [13, 10,  6,  3],
                             [ 8,  5,  4,  1]], dtype=int8)
ic| listcurrentstate: array([[ 2, 15,  0, 12],
                             [13, 11, 14,  6],
                             [ 8,  1,  9,  5],
                             [10,  7,  4,  3]], dtype=i